In [10]:
import pandas as pd

# CSV 파일을 불러옵니다.
df = pd.read_csv('./paths/dbscan/test_files_3_dbscan.csv')

# 'Class' 열에서 0의 개수를 셉니다.
count_Normal = (df['Label'] == 0).sum()
count_AF = (df['Label'] == 1).sum()
count_Other = (df['Label'] == 2).sum()

count_Total = count_Normal + count_AF + count_Other

print(f"Class N: {count_Normal}")
print(f"Class A: {count_AF}")
print(f"Class O: {count_Other}")
print(f"Total: {count_Total}") 

Class N: 30172
Class A: 6384
Class O: 18124
Total: 54680


In [54]:
print(df['File Path'].iloc[1])

/mnt/HDD_2/ECG_Dataset/Physionet_CinC_2017/CATNet_Dataset/1s/A08269_S64.mat


In [11]:
print((30172 / 54680))
print((6384 / 54680))
print((18124 / 54680))

0.5517922457937089
0.11675201170446232
0.3314557425018288


In [47]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('./paths/train_files.csv')

# 'Label' 열의 데이터 타입 변환
df['Label'] = df['Label'].astype('int64')

# 변경된 데이터프레임 확인
print(df.dtypes)

# 수정된 DataFrame을 새로운 CSV로 저장 (원하는 경우)
df.to_csv('./paths/train_files_updated.csv', index=False)

0         0
1         2
2         1
3         2
4         2
         ..
347805    0
347806    1
347807    2
347808    0
347809    1
Name: Label, Length: 347810, dtype: object


/tmp/ipykernel_11907/405278182.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./paths/train_files.csv')


ValueError: invalid literal for int() with base 10: 'Label'

In [44]:
print(df['File Path'].dtype)

object


# 여기부터 실행할 것

In [1]:
import os
import random
import csv
import pandas as pd

# .mat 파일들이 저장된 디렉토리 경로를 지정합니다.
mat_files_dir = "/mnt/HDD_2/ECG_Dataset/Physionet_CinC_2017/CATNet_Dataset/1s"  # 여기에 실제 경로를 입력하세요.
# 'annotation.csv' 파일을 불러옵니다.
annotations_path = "/mnt/HDD_2/ECG_Dataset/Physionet_CinC_2017/CATNet_Dataset/segments_ann_CATNet.csv"  # 실제 경로를 입력하세요.

# 디렉토리에서 모든 .mat 파일을 가져옵니다.
mat_files = [f for f in os.listdir(mat_files_dir) if f.endswith('.mat')]

# 파일 경로를 무작위로 섞습니다.
random.seed(42)
random.shuffle(mat_files)


all_files = mat_files


# 파일을 80%는 학습용, 20%는 테스트용으로 나눕니다.
split_index = int(0.8 * len(mat_files))
train_val_files = mat_files[:split_index]
test_files = mat_files[split_index:]

# train 파일을 다시 85%는 학습용, 15%는 검증용으로 나눕니다.
val_split_index = int(0.85 * len(train_val_files))
train_files = train_val_files[:val_split_index]
val_files = train_val_files[val_split_index:]

annotations = pd.read_csv(annotations_path)

# 라벨 인코딩: 'N' -> 0, 'A' -> 1, 'O' -> 2
label_mapping = {'N': 0, 'A': 1, 'O': 2}
annotations['Encoded_Label'] = annotations['Class'].map(label_mapping)

# 파일 경로와 매칭되는 라벨을 찾는 함수
def get_label(file_name):
    # 파일 이름만 추출하여 '.mat' 파일 확장자를 제거
    base_name = os.path.splitext(file_name)[0]
    #print(base_name)
    label_row = annotations[annotations['Filename'] == base_name]  # 파일 이름과 매칭되는 라벨 찾기
    if not label_row.empty:
        return label_row['Encoded_Label'].values[0]
    else:
        return None  # 해당 파일이 annotation.csv에 없으면 None 반환

# 파일 경로를 CSV로 저장하는 함수 (라벨 포함, 체크포인트 지원)
def save_to_csv_with_checkpoint(file_list, csv_path, start_idx=0):
    with open(csv_path, mode='a', newline='') as file:  # 'a' 모드로 열어 이어쓰기
        writer = csv.writer(file)
        if start_idx == 0:  # 처음 실행할 때만 헤더 추가
            writer.writerow(['File Path', 'Label'])

        for idx, file_name in enumerate(file_list[start_idx:], start=start_idx):
            label = get_label(file_name)
            if label is not None:  # 라벨이 존재할 때만 저장
                writer.writerow([os.path.join(mat_files_dir, file_name), label])
            if idx % 100 == 0:  # 100개마다 저장 체크포인트 기록
                print(f'{idx}개의 파일이 처리되었습니다.')


In [ ]:
# 학습용 파일 경로와 라벨을 train_files.csv에 저장합니다.
save_to_csv_with_checkpoint(train_files, './paths/train_files_2.csv')

save_to_csv_with_checkpoint(val_files, './paths/val_files_2.csv')

# 테스트용 파일 경로와 라벨을 test_files.csv에 저장합니다.
save_to_csv_with_checkpoint(test_files, './paths/test_files_2.csv')

print("파일 경로와 라벨이 각각 train_files.csv와 test_files.csv에 저장되었습니다.")

In [2]:
# 학습용 파일 경로와 라벨을 train_files.csv에 저장합니다.
save_to_csv_with_checkpoint(all_files, './paths/all_files.csv')

print("파일 경로와 라벨이 각각 train_files.csv와 test_files.csv에 저장되었습니다.")

0개의 파일이 처리되었습니다.
100개의 파일이 처리되었습니다.
200개의 파일이 처리되었습니다.
300개의 파일이 처리되었습니다.
400개의 파일이 처리되었습니다.
500개의 파일이 처리되었습니다.
600개의 파일이 처리되었습니다.
700개의 파일이 처리되었습니다.
800개의 파일이 처리되었습니다.
900개의 파일이 처리되었습니다.
1000개의 파일이 처리되었습니다.
1100개의 파일이 처리되었습니다.
1200개의 파일이 처리되었습니다.
1300개의 파일이 처리되었습니다.
1400개의 파일이 처리되었습니다.
1500개의 파일이 처리되었습니다.
1600개의 파일이 처리되었습니다.
1700개의 파일이 처리되었습니다.
1800개의 파일이 처리되었습니다.
1900개의 파일이 처리되었습니다.
2000개의 파일이 처리되었습니다.
2100개의 파일이 처리되었습니다.
2200개의 파일이 처리되었습니다.
2300개의 파일이 처리되었습니다.
2400개의 파일이 처리되었습니다.
2500개의 파일이 처리되었습니다.
2600개의 파일이 처리되었습니다.
2700개의 파일이 처리되었습니다.
2800개의 파일이 처리되었습니다.
2900개의 파일이 처리되었습니다.
3000개의 파일이 처리되었습니다.
3100개의 파일이 처리되었습니다.
3200개의 파일이 처리되었습니다.
3300개의 파일이 처리되었습니다.
3400개의 파일이 처리되었습니다.
3500개의 파일이 처리되었습니다.
3600개의 파일이 처리되었습니다.
3700개의 파일이 처리되었습니다.
3800개의 파일이 처리되었습니다.
3900개의 파일이 처리되었습니다.
4000개의 파일이 처리되었습니다.
4100개의 파일이 처리되었습니다.
4200개의 파일이 처리되었습니다.
4300개의 파일이 처리되었습니다.
4400개의 파일이 처리되었습니다.
4500개의 파일이 처리되었습니다.
4600개의 파일이 처리되었습니다.
4700개의 파일이 처리되었습니다.
4800개의 파일이 처리되었습니다.
4900개의 파일이 처리되었습니다.
5000개의 파일이 처

# All Files Train / Validation / Test Split

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터 로드
all_files = pd.read_csv("./paths/dbscan/all_files_dbscan.csv")

# 1. Train-Test Split (8:2)
train_data, test_data = train_test_split(all_files, test_size=0.2, stratify=all_files['Label'], random_state=42)

# 2. Train-Validation Split (train의 15%)
train_data, val_data = train_test_split(train_data, test_size=0.15, stratify=train_data['Label'], random_state=42)

# 3. 각 데이터 세트를 별도 CSV 파일로 저장
train_data.to_csv("./paths/dbscan/train_files_3_dbscan.csv", index=False)
val_data.to_csv("./paths/dbscan/val_files_3_dbscan.csv", index=False)
test_data.to_csv("./paths/dbscan/test_files_3_dbscan.csv", index=False)

print("Train, Validation, Test splits have been saved.")

Train, Validation, Test splits have been saved.
